In [1]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

d:\Dhanwis\RAG_chatbot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Step 1: Load PDF
loader = PyPDFLoader(r"D:\Dhanwis\RAG_chatbot\college_data.pdf")
pages = loader.load()

In [3]:
# Step 2: Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=50)
documents = splitter.split_documents(pages)

In [4]:
# Step 3: Embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vectorstore = FAISS.from_documents(documents, embedding_model)

C:\Users\user\AppData\Local\Temp\ipykernel_19264\1839621465.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [5]:
# Step 4: Load a small LLM locally using Transformers
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    device_map="auto",  # Automatically places model layers on available devices (CPU/GPU)
    torch_dtype="auto"  # Uses the best supported dtype
)

Some parameters are on the meta device because they were offloaded to the cpu and disk.


In [6]:
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=256)

Device set to use cpu


In [7]:
# Step 5: Wrap in LangChain
llm = HuggingFacePipeline(pipeline=pipe)
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())

C:\Users\user\AppData\Local\Temp\ipykernel_19264\3938560957.py:2: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [8]:
# Step 6: Ask questions
while True:
    query = input("Ask something (or 'exit'): ")
    if query.lower() == "exit":
        break
    print("Answer:", qa_chain.run(query))

C:\Users\user\AppData\Local\Temp\ipykernel_19264\3286432490.py:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print("Answer:", qa_chain.run(query))
Token indices sequence length is longer than the specified maximum sequence length for this model (659 > 512). Running this sequence through the model will result in indexing errors


Answer: yes
Answer: yes
Answer: yes
Answer: Python for Beginners
Answer: python for beginners
Answer: yes
Answer: Python for Beginners
Answer: Dr. Rajesh Kumar
Answer: Merit-Based Scholarship
Answer: Minimum 90% in previous academic year
Answer: 123 Tech Park, Silicon Valley, CA
